# This is a recreation of the work in: Model Predictive Control for integrated lateral stability, traction/braking control, and rollover prevention of electric vehicles 
By: Mansour Ataei, Amir Khajepour & Soo Jeon

In [ ]:
# Run this cell only if you are using Google Colab. 

# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !pip install mosek
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64
  !pip install ttictoc

     |████████████████████████████████| 9.7 MB 4.7 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.4 MB 4.7 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123941 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) 

In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    !pip install casadi
    helper.install_idaes()
    helper.install_ipopt()

--2022-11-09 03:25:19--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6456 (6.3K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   6.30K  --.-KB/s    in 0s      

2022-11-09 03:25:19 (70.6 MB/s) - ‘helper.py’ saved [6456/6456]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.2 MB 37 kB/s 
Installing idaes via pip...
idaes was successfully installed
Ipopt found! No need to install.
ipopt was successfully installed


The Dugoff model for a typical tire is Dugoff 1970, Wong, 1978

$$
\begin{aligned}
&F_x=-\frac{C_s s}{1-s} f(\lambda) \\
&F_y=-\frac{C_\alpha \tan \alpha}{1-s} f(\lambda)
\end{aligned}
$$
\\
$$
\begin{aligned}
&\lambda=\frac{\mu F_z\left(1-\epsilon v \sqrt{s^2+\tan ^2 \alpha}\right)(1-s)}{2 \overline{c_s^2 s^2+C_\alpha^2 \tan ^2 \alpha}} \\
\\
&f(\lambda)= \begin{cases}X(2-\lambda) & \text { for } \lambda<1 \\
1 & \text { for } \lambda>1\end{cases}
\end{aligned}
$$

$F_x=$ longitudinal force \\
$F_y=$ side force

$\mathrm{s}=$ longitudinal slip \\
$\alpha=$ tire slip angle \\
$C_s=$ cornering stiffness \\
$C_\alpha=$ longitudinal stiffens \\
$F_z=$ vertical load \\
$v=$ vehicle speed \\
$\mu=$ coefficient of road adhesion \\
$\epsilon=$ adhesion reduction coefficient

In this section, we use $\alpha$ to denote the tire slip angle


============================================================================

MODEL USED

given by (Guntur and Sankar, 1980, Dugoff, 1969)
$$
F_x=C_\sigma \frac{\sigma_x}{1+\sigma_x} f(\lambda)
$$
and the lateral tire force is given by $\quad F_y=C_\alpha \frac{\tan (\alpha)}{1+\sigma_x} f(\lambda)$
where $\lambda$ is given by
$$
\lambda=\frac{\mu F_z\left(1+\sigma_x\right)}{2\left\{\left(C_\sigma \sigma_x\right)^2+\left(C_\alpha \tan (\alpha)\right)^2\right\}^{1 / 2}}
$$
and
$f(\lambda)=(2-\lambda) \lambda$ if $\lambda<1$
$f(\lambda)=1$ if $\lambda \geq 1$
$F_z$ is the vertical force on the tire while $\mu$ is the tire-road friction coefficient.




In [ ]:
def DugoffLinearizer(slip_angle,slip_ratio, ca, cs, mu, Fz):
  x = sym.Symbol('x')
  lamb_bar = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)**2+(ca*slip_angle)**2)**0.5)
  lamb = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)**2+(ca*x)**2)**0.5)

  if lamb_bar < 1:
    f = lamb*(2-lamb)
  else:
    f = 1

  Fy = ca*x*f[0]/(1+slip_ratio)
  Fy_bar= Fy[0].subs(x,slip_angle)
  dfy_alph = sym.diff(x,Fy)
  c_bar = dfy_alph.evalf(subs = slip_angle)

  return Fy_bar, c_bar, slip_angle

In [ ]:
import numpy as np
from scipy.linalg import block_diag
import sympy as sym
import casadi as cas

**Optimal_Control_partI.pdf**
2. Linear Quadratic Optimal Control
2.1 Batch Approach Solution

Starting with $x_0=x(0)$, we have $x_1=A x(0)+B u_0$, and $x_2=A x_1+B u_1=A^2 x(0)+A B u_0+B u_1$, by substitution for $x_1$, and so on. Continuing up to $x_N$ we obtain:
$$
\left[\begin{array}{c}
x_0 \\
x_1 \\
\vdots \\
\vdots \\
x_N
\end{array}\right]=\left[\begin{array}{c}
I \\
A \\
\vdots \\
\vdots \\
A^N
\end{array}\right] x(0)+\left[\begin{array}{cccc}
0 & \cdots & \cdots & 0 \\
B & 0 & \cdots & 0 \\
A B & B & \cdots & 0 \\
\vdots & \ddots & \ddots & 0 \\
A^{N-1} B & \cdots & A B & B
\end{array}\right]\left[\begin{array}{c}
u_0 \\
u_1 \\
\vdots \\
\vdots \\
u_{N-1}
\end{array}\right]
$$

The equation above can be represented as
$$
\mathcal{X} \triangleq \mathcal{S}^x x(0)+\mathcal{S}^u U_0
$$

In [ ]:
def Sx_Su_Sw_Sd(A, B, E, D, N):
  nX = np.size(A,0)
  nU = np.size(B,1)
  nE = np.size(E,1)

  Sx = np.eye(nX)
  A_tmp = A
  for i in range(N):
      Sx = np.vstack((Sx, A_tmp))
      A_tmp = A_tmp @ A
      
  SxB = Sx @ B
  Su = np.zeros((nX*(N+1),nU*N))
  
  for j in range(N):
      Su_tmp = np.vstack((np.zeros((nX, nU)), SxB[:-nX,:]))
      Su[:, j:j+8] = Su_tmp.reshape(Su_tmp.shape[0],8 )
      SxB = Su_tmp

  SxE = Sx @ E
  Sw = np.zeros((nX*(N+1),nE*N))
  for k in range(N):
      Sw_tmp = np.vstack((np.zeros((nX, nE)), SxE[:-nX,:]))
      Sw[:, j:j+8] = Sw_tmp.reshape(Sw_tmp.shape[0],8 )
      SxE = Sw_tmp

  Sd = np.zeros((nX,1))
  D_tmp = D
  for i in range(N):
      Sd = np.vstack((Sd, D_tmp))
      D_tmp = A @ D_tmp
  return Sx, Su, Sw, Sd

In [ ]:
def CFTOC_frompaper(slip_angle,slip_ratio,ca,cs,mu,fz0,u,reff,delta,tf,tr,lf,lr,ms,hs,hr,hu,kphi,g,ixx,m,cphi,iw,x0,delta_d,kus_d,muy,mux,alphamax,lambmax,RIc,W0,N,Qmax,fy0):
  nX = 8
  nU = 8
  x0.reshape(8,1)
  
  # Corner forces
  fy1, ca1, a1 = DugoffLinearizer(slip_angle[0],slip_ratio[0], ca[0], cs[0], mux, fz0[0])
  fy2, ca2, a2 = DugoffLinearizer(slip_angle[1],slip_ratio[1], ca[1], cs[1], mux, fz0[1])
  fy3, ca3, a3 = DugoffLinearizer(slip_angle[2],slip_ratio[2], ca[2], cs[2], mux, fz0[2])
  fy4, ca4, a4 = DugoffLinearizer(slip_angle[3],slip_ratio[3], ca[3], cs[3], mux, fz0[3])

  B11 = np.array([[0,0,0,0], [-ca1/u, -lf*ca1/u, 0, 0]])
  B21 = np.array([[1/reff, 0],[0, ca1]])
  D11 = np.array ([0, fy1-ca1*a1])

  B12 = np.array([[0,0,0,0], [-ca2/u, -lf*ca2/u, 0, 0]])
  B22 = np.array([[1/reff, 0],[0, ca2]])
  D12 = np.array ([0, fy2-ca2*a2])

  B13 = np.array([[0,0,0,0], [-ca3/u, -lr*ca3/u, 0, 0]])
  B23 = np.array([[1/reff, 0],[0, ca3]])
  D13 = np.array ([0, fy3-ca3*a3])

  B14 = np.array([[0,0,0,0], [-ca4/u, -lr*ca4/u, 0, 0]])
  B24 = np.array([[1/reff, 0],[0, ca4]])
  D14 = np.array ([0, fy4-ca4*a4])

  # Load Actuator Reconfiguration Matrix
  Tw1 = np.array([[1,0],[0,0]])
  Tw2 = np.array([[1,0],[0,0]])
  Tw3 = np.array([[1,0],[0,0]])
  Tw4 = np.array([[1,0],[0,0]])

  #Reconfiguration (Mapping) Matricies: Delta1 input
  Lw1 = np.array ([[np.cos(delta[0]), -np.sin(delta[0])],[np.sin(delta[0]), np.cos(delta[0])]])
  Lw2 = np.array ([[np.cos(delta[1]), -np.sin(delta[1])],[np.sin(delta[1]), np.cos(delta[1])]])
  Lw3 = np.array ([[np.cos(delta[2]), -np.sin(delta[2])],[np.sin(delta[2]), np.cos(delta[2])]])
  Lw4 = np.array ([[np.cos(delta[3]), -np.sin(delta[3])],[np.sin(delta[3]), np.cos(delta[3])]])

  #Mapping Matrix from corner to CG forces
  Lc = np.array([[1,0,1,0,1,0,1,0],[0,1,0,1,0,1,0,1], [-tf/2,lf,tf/2,lf,-tr/2,-lr,tr/2,-lr]])

  #Vehicle Body Dynamics
  Af = [[0,-u, ms*hs*(kphi-ms*g*hs)/(m*ixx+ms**2*hs**2), ms*hs*cphi/(m*ixx-ms**2*hs**2)], [0,0,0,0],[0,0,0,0],[0,0,-m*(kphi-ms*g*hs)/(m*ixx-ms**2*hs**2), -m*cphi/(m*ixx-ms**2*hs**2)]]
  Bf = [[0,ixx/(m*ixx-ms**2*hs**2),0], [0,0,1/Izz],[0,0,0], [0,ms*hs/(m*ixx-ms**2*hs**2),0]]


  #Compilation
  B1 = np.hstack((B11.T, B12.T, B13.T, B14.T)).T
  B2 = block_diag(B21,B22,B23,B24)
  D1 = np.hstack((D11,D12,D13,D14)).T
  Tw = block_diag(Tw1,Tw2,Tw3,Tw4)
  Lw = block_diag(Lw1,Lw2,Lw3,Lw4)

  #Xb_dot = AXb +EW +BU +D
  Ab = Af+Bf@Lc@Lw@B1
  Eb = Bf@Lc@Lw@B2
  Bb = Bf@Lc@Lw@B2@Tw
  Db = Bf@Lc@Lw@D1

  #wheel:
  Aw = np.zeros((4,4))
  Ew = np.zeros((4,8))
  Bw = 1/iw * np.array([[1,0,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,1,0]])
  Dw = np.zeros((1,4))
  
  lambd =np.zeros((4,1))
  for k in range(4):
    lambd[k] = (reff*x0[k+4]-u)/np.maximum(u,reff*x0[k+4])


  A = block_diag(Ab, Aw)
  E = np.hstack((Eb.T, Ew.T)).T
  B = np.hstack((Bb.T, Bw.T)).T
  D = np.hstack((Db.T, Dw[0].T)).T 
  D = D.reshape(8,1)
  
################################################################################
  ## CONTROL OBJECTIVES ##
  # Yaw   Rate
  rb = u*delta_d/(l+kus_d*u**2)
  rmax = muy*g/u 
  rd = np.sign(delta_d)*np.min([np.abs(rb), rmax])

  # Lateral Stability

  # Rollover Prevention
  C1 = 4/m/g/(tf+tr)*(kphi*(1+(ms*hr+mu*hu)/(ms*hs))-(ms*hr+mu*hu)*g)
  C2 = 2*cphi/(m*T*g)*(1+(ms*hr+mu*hu)/(ms*hs))
  #Impose rollover index contraint

  wub = np.zeros((4,1))
  wlb =np.zeros((4,1))
  Rw = reff
  #Wheel spin constraint:
  wub[0] = u/Rw+lambmax*np.max([u/Rw, x0[4]])
  wub[1] = u/Rw+lambmax*np.max([u/Rw, x0[5]])
  wub[2] = u/Rw+lambmax*np.max([u/Rw, x0[6]])
  wub[3] = u/Rw+lambmax*np.max([u/Rw, x0[7]])

  wlb[0] = u/Rw-lambmax*np.max([u/Rw, x0[4]])
  wlb[1] = u/Rw-lambmax*np.max([u/Rw, x0[5]])
  wlb[2] = u/Rw-lambmax*np.max([u/Rw, x0[6]])
  wlb[3] = u/Rw-lambmax*np.max([u/Rw, x0[7]])

  xd = np.array([0,rd,0,0,u/reff,u/reff,u/reff,u/reff])
  xd.reshape(8,1)
  RI = C1*x0[2]+C2*x0[3]
  Qw =np.zeros((4,1))
  ns = 2
  gamma_s = 5
  for i in range(4):
    Qw[i] = gamma_s*(np.abs(lambd[i])+(1-lambmax))**ns
  Q_RI = 5*(np.abs(RI)+(1-RIc))**10
  Q_r = 10
  Q_v = 7

  #CFTOC With Subs
  Ad = Ts*A + np.eye(np.size(A,0))
  Bd = Ts*B
  Ed = Ts*E
  Dd = Ts*D
  Sx, Su, Sw, Sd = Sx_Su_Sw_Sd(Ad, Bd, Ed, Dd, N)
  

  # We need a weighting matrix Q. I would say emphasis on tracking lateral position, then heading, then lateral velocity, with little concern for roll charact.
  Q = np.array([Q_v,Q_r,Q_RI,Q_RI,Qw[0][0],Qw[1][0],Qw[2][0],Qw[3][0]])
  PN = Q
  Q_temp = np.kron(np.eye(N),np.diag(Q))
  Qbar  = block_diag(Q_temp,np.diag(PN))
  ##R = np.array(np.ones((8,1)))
  ##R_temp = np.kron(np.eye(N), np.diag(R))
  Rbar = np.eye(nU*N)
  ##Rbar = block_diag(R_temp)

  Wbar = np.kron(np.ones((1,N)), W0) #W0 recieved from vehicle, vector of inputs at time zero #### check
  Xdbar = np.kron(np.ones((1,N+1)), xd)

  H = Su.T@Qbar@Su + Rbar
  F1 = Sx.T @Qbar @ Su
  F2 = Sw.T @ Qbar @ Su
  F3 = Sd.T@Qbar@Su
  F4 = - Qbar@Su

  # Finally Cost Matricies:

  P = H

  q = x0.T@F1+Wbar@F2 +F3+ Xdbar@F4 #x0 from vehicle, xdes from path planner

  Ax = np.vstack((np.array([
      [0,1,0,0,0,0,0,0],
      [0,-1,0,0,0,0,0,0],
      [-1/u, lr/u,0,0,0,0,0,0],
      [1/u, -lr/u,0,0,0,0,0,0],
      [0,0,C1,C2,0,0,0,0],
      [0,0,-C1,-C2,0,0,0,0]]),
      np.hstack((np.zeros((4,4)), np.eye(4))), 
      np.hstack((np.zeros((4,4)), -np.eye(4)))
  ))
  bx = np.array([rmax, rmax, alphamax,alphamax,RIc,RIc, wub[0],wub[1],wub[2],wub[3], wlb[0],wlb[1],wlb[2],wlb[3]]).reshape(14,1)

  fx0_p=np.zeros((4,1))
  for i in range(4):
    fx0_p[i] = mux*fz0[i]*np.sqrt(1-(fy0[i]/(muy*fz0[i]))**2)
  Au = np.array([[1,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,1,0]])
  Au3 = np.array([[0,1,0,-1,0,0,0,0],[0,-1,0,1,0,0,0,0]])
  Au = np.vstack((Au,-Au,Au,-Au,Au3))
  bu_fp = fx0_p.T-np.array([W0[0],W0[2],W0[4],W0[6]])
  bu_fp = bu_fp.reshape(4,1)

  bu_sp = fx0_p.T+np.array([W0[0],W0[2],W0[4],W0[6]])
  bu_sp = bu_sp.reshape(4,1)
  butp = Qmax*np.ones((1,4)) - np.array([W0[0],W0[2],W0[4],W0[6]])
  butp = butp.reshape(4,1)
  bufp = Qmax*np.ones((1,4)) + np.array([W0[0],W0[2],W0[4],W0[6]])
  bufp = bufp.reshape(4,1)


  bu = np.vstack((bu_fp,bu_sp,butp,bufp,0,0))


  G0_firstpart= block_diag(np.kron(np.eye(N),Au))
  temp = np.kron(np.ones((N+1,N+1)), Ax)
  G0_sp = temp@Su
  G0 = np.vstack((G0_firstpart,G0_sp))
  print(np.kron(np.ones((N,1)),bu).shape)
  print(np.kron(np.ones((N+1,1)),bx).shape)

  w0 = np.vstack((np.kron(np.ones((N,1)),bu), np.kron(np.ones((N+1,1)),bx)))


  e0_sp = -Ax
  Ax_tmp = -Ax@A
  for i in range(N):
      e0_sp = np.vstack((e0_sp, Ax_tmp))
      Ax_tmp = Ax_tmp @ A
  e0_fp = np.zeros((N*16, 8)) ##2 length of Bu *n
  
  E0 = np.vstack((e0_fp,e0_sp))

  A = G0
  b = E0*x0+w0

  z = cas.SX.sym('z', nU*N,1)
  print(H)
  print(q)
  print(A)

  nlp = {'z':z , 'f': cas.mtimes(z.T,cas.mtimes(H,z))+cas.mtimes(q,z), 'g': A}
  opts = {'ipopt.print_level': 0, 'ipopt.sb':'yes','print_time':0}
  S = cas.nlpsol('solver','ipopt',nlp,opts)
  sol = cas.solver(lbg=-np.inf, ubg=b)
  uopt = sol['z']
  JOpt = sol['f']

  return uopt, JOpt

In [ ]:
#Model Parameters from 
ms = 1590
mu = 270
m = ms+mu
T = 1.575 
tf = T
tr = T
hcg = 0.72
l = 2.95
lf = 1.18
lr = 1.77
ixx = 894.4
Izz = 2687.1
hs = 0.57 
hu = 0.2
hr = 0.15
kphi = 189506
cphi = 6364
reff = 0.393
iw = 1.1
g = 9.81

#Case 1
mux = 1
muy = 1
u = 80/3.6
N = 10
Ts = 0.02 
lambmax= 0.1
alphamax = 6/180*np.pi
Qmax = 1600
RIc = 0.7


kus_d = 0 #####

delta_d = 0.1####
cs = 80000*np.ones((4,1)) ####
ca = 47275*np.ones((4,1)) #####
delta = [0,0,0,0]
x0 = np.array([0,0,0,0,0,0,0,0])
slip_angle = [0.1,0.1,0.1,0.1]
slip_ratio = [0,0,0,0]
fz0 = m*g/4*np.ones((4,1))
W0 = [0,0,0,0,0,0,0,0]
fy0 = np.zeros((4,1))

uopt, jopt = CFTOC_frompaper(slip_angle,slip_ratio,ca,cs,mu,fz0,u,reff,delta,tf,tr,lf,lr,ms,hs,hr,hu,kphi,g,ixx,m,cphi,iw,x0,delta_d,kus_d,muy,mux,alphamax,lambmax,RIc,W0,N,Qmax,fy0)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:161: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


(180, 1)
(154, 1)


ValueError: ignored